# Recherche et alignement, outils de cartographie et de correspondance de la basea de données Gallica via l'API:



In [ ]:
#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive.

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''


## Installation des bibliothèques et connexion au compte Google Drive

!pip install utils
from google.colab import drive
import json
from openpyxl import load_workbook
import pandas as pd
import re
import requests
!pip install langdetect
from langdetect.lang_detect_exception import LangDetectException
import shutil
!pip install xmltojson
import xmltodict
import urllib
import urllib.request, urllib.error, urllib.parse
import xmltojson
from langdetect import detect
import os
import xml.etree.ElementTree as ET
import lxml
from collections import OrderedDict
from lxml import etree
!pip install unidecode==1.2.0
import unidecode
import folium
import unicodedata
from urllib.error import HTTPError
from re import match
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import string

drive.mount('/content/drive/')


#@markdown Lors du lancement de la cellule, la connexion vers le Google Drive sera nécessaire, les étapes à suivre sont:

#@markdown - Indiquer le chemin absolu vers le dossier de travail contenu sur le Google Drive (ici, le dossier de travail se nomme "FabLab")
chemin_absolu_vers_le_dossier_de_travail = '/content/drive/My Drive/FabLab/' #@param {type:"string"}



#@markdown - En dessous de la cellule, un lien bleu va apparaître après 'Go to this URL in a browser'. Cliquer dessus, un nouvel onglet du navigateur va s'ouvrir.

#@markdown - Sur ce nouvel onglet, cliquer sur le compte Google auquel le Drive est associé. Cliquer ensuite sur "Connexion" puis sur la petite icône en forme de carré pour copier le lien.

#@markdown - De retour sur le notebook de travail, coller le lien dans la case en dessous de la cellule après "Enter your authorization code:" puis appuyer sur la touche "Enter" du clavier. La cellule va finir de l'exécuter normalement.

os.chdir(chemin_absolu_vers_le_dossier_de_travail)


In [ ]:
#@title Recherche de données correspondantes à partir d'un terme dans la base de données Gallica.

#@markdown

#@markdown Indiquer ici le terme à chercher dans la base de données GALLICA: possibilité de ne mettre qu'un titre (d'oeuvre) ou un artiste, ou les deux! Il est aussi possiblé de ne séléctionner qu'un type d'objets en particuliers; mettre "tout" pour n'en séléctionner aucun en particulier.

#@markdown Accents non reconnus ! Exemple, pour chercher : Molière, mettre : Moliere


#@ Les valeurs possibles sont monographie, carte, image, fascicule (pour les publications en série), manuscrit, partition, sonore (pour les documents son), objet et vidéo.
titre_a_chercher = "guerre d'espagne" #@param {type:"string"}
auteur_a_chercher = "" #@param {type:"string"}
type_a_chercher = "tout" #@param ["tout", "monographie", "carte", "image", "fascicule", "manuscrit", "partition", "sonore", "objet", "vidéo"]



os.chdir(chemin_absolu_vers_le_dossier_de_travail)
if titre_a_chercher != "":
  os.makedirs(titre_a_chercher, exist_ok=True)
  os.chdir(titre_a_chercher)
 
elif auteur_a_chercher != "":
  os.makedirs(auteur_a_chercher, exist_ok=True)
  os.chdir(auteur_a_chercher)


chemin_dossier_recherche = os.path.abspath(os.getcwd())
lien_base_recherche = "https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve%22&maximumRecords=49&query="
lien_de_recherche_gallica = None

### RECHERCHE GALLICA
if titre_a_chercher != "":
  if ' ' in titre_a_chercher:
    titre_a_chercher = re.sub(r' ', '%20', titre_a_chercher)
    titre_a_chercher = ''.join("\""+titre_a_chercher+"\"")
    titre_pour_lien = "".join("dc.title%20adj%20%22"+ titre_a_chercher+"%22")
  else:
    titre_a_chercher=titre_a_chercher
    titre_pour_lien = "".join("dc.title%20any%20"+ titre_a_chercher)
  lien_de_recherche_gallica = "".join(lien_base_recherche+ titre_pour_lien)


if auteur_a_chercher != "":
  if ' ' in auteur_a_chercher:
    auteur_a_chercher = re.sub(r' ', '%20', auteur_a_chercher)
  else:
    auteur_a_chercher=auteur_a_chercher
  auteur_pour_lien = "".join("dc.creator%adj%20\""+ auteur_a_chercher + "\"")
  if lien_de_recherche_gallica is None:
    lien_de_recherche_gallica = "".join(lien_base_recherche + auteur_pour_lien)
  else:
    lien_de_recherche_gallica = "".join(lien_de_recherche_gallica + "%20&%20"+auteur_pour_lien)

if type_a_chercher is not "tout":
  type_pour_lien = "".join("%20and%20dc.type%20any%20"+type_a_chercher)
  lien_de_recherche_gallica = "".join(lien_de_recherche_gallica + type_pour_lien)

print(lien_de_recherche_gallica)
request = urllib.request.Request(lien_de_recherche_gallica)
try:
  content = urllib.request.urlopen(request)
  parse1 = BeautifulSoup(content,'html.parser')
  dict_final = []
  my =xmltodict.parse(parse1.prettify())
  dict_final.append(my)
except HTTPError as he:
  print('Pas de téléchargement possible pour la base Gallica avec cette recherche')

for key, value in my.items():
    for key2, value2 in value.items():
        if key2 == 'srw:numberofrecords':
          nbr_results = value2
          print("Le nombre de résultats poour la base Gallica est de : " + str(nbr_results))
          if int(nbr_results) > 50:
            multiples = [n for n in range(1, int(nbr_results)) if n % 50 == 0]
            nbr_recherche_depart = 0
            for multiple in multiples:
              nbr_recherche_depart =nbr_recherche_depart+50
              lien_de_recherche_gallica2 = "".join(lien_de_recherche_gallica+'&startRecord='+str(nbr_recherche_depart))
              request = urllib.request.Request(lien_de_recherche_gallica2)
              try:
                content = urllib.request.urlopen(request)
                parses = BeautifulSoup(content,'html.parser')
                my3 =xmltodict.parse(parses.prettify())
                dict_final.append(my3)
              except HTTPError  as hto:
                print("Erreur de connexion au serveur Gallica, ré-essayer!")
nom_du_fichier_json_base_gallica = 'donnees_base_gallica_guerre_espagne.json' #@param {type:"string"}
with open(nom_du_fichier_json_base_gallica, 'w') as pi:
  json.dump(dict_final, pi, indent=4, ensure_ascii=False)
print("Fichier de données enregistré!")


#@markdown Il est possible de jeter un oeil au fichier créé qui contient déjà les informations "brutes" tirées dde la base de données



In [ ]:
### Base GALLICA
#@markdown Indiquer ici le nom du dossier dans lesquelles les images de la base Gallica vont se télécharger:
dossier_base_gallica = 'images_base_gallica_guerre_espagne' #@param {type:"string"}
new_gallica = {}
#chemin_fichier2 = "".join(chemin_dossier+'/'+nom_du_fichier_json_base_gallica)
with open(nom_du_fichier_json_base_gallica, 'r') as ret:
  gallica = json.load(ret)
print("Les images sont en train de se télécharger...")
for ele in gallica:
  for key, value in ele.items():
    for key2, value2 in value.items():
      if key2 == 'srw:records':
        for key3, value3 in value2.items():
          if key3 == 'srw:record':
            for ele2 in value3:
              for key, value in ele2.items():
                if key == 'srw:recorddata':
                  for key2, value2 in value.items():
                    for key3, value3 in value2.items():
                      if key3 == 'dc:title':
                        le_titre = str(value3)
                        le_nouveau_titre = re.sub(r"[\['\]]", '', le_titre)
                        new_gallica[le_nouveau_titre] = {}
                      if key3 == 'dc:date':
                        date = value3
                      if key3 == 'dc:creator':
                        auteur = value3
                      if key3 == 'dc:type':
                        types = value3
                      if key3 == 'dc:subject':
                        subject = value3
                      if key3 == 'dc:identifier':
                        if type(value3) == list:
                          value3 = value3[1]
                        identifiant = value3[27:]
                        lien = 'http://gallica.bnf.fr/iiif/ark:' + identifiant + '/f1/full/3000/0/native.jpg'
                        new_gallica[le_nouveau_titre] = {'Date': date, 'Auteur':auteur, 'Type': types, 'Sujet': subject, 'Lien': lien}
os.chdir(chemin_dossier_recherche)
#@markdown Indiquer ici le nom du fichier de données nettoyé contenant uniquement les informations essentielles, base Gallica. Le fichier est au format json.
print('Création du nouveau fichier nettoyé pour la base Gallica.')
nom_du_fichier_nettoyé_base_gallica = 'base_Gallica_venus_cartes_final.json' #@param {type:"string"}
with open(nom_du_fichier_nettoyé_base_gallica, 'w') as pl:
  json.dump(new_gallica, pl, indent=4, ensure_ascii=False)

if not os.path.exists(dossier_base_gallica):
  os.mkdir(dossier_base_gallica)
os.chdir(dossier_base_gallica)
for rekey, revalue in new_gallica.items():
  rekey = re.sub(r"[/;:']", "_", rekey)
  nom_image = "".join(rekey + '.jpg')
  nom_imageb = re.sub(r"[/;:' ]", "_", nom_image)
  nom_imagec = "".join(nom_imageb +'_'+ark+ '.jpg')
  for rekey2, revalue2 in revalue.items():
    if rekey2 == 'Lien':
      if ' ' in revalue2:
        revalue2 = re.sub(r' ', '%20', revalue2)
      try:
        urllib.request.urlretrieve(revalue2, nom_imagec)
      except (OSError, HTTPError, urllib.error.ContentTooShortError, UnicodeEncodeError) as http:
        pass